In [17]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from icecream import ic
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from itertools import accumulate
import functools


In [18]:
logging.basicConfig(level=logging.DEBUG)

## Nearest Neighbor (Greedy)

In [19]:
def nearest_neighbor_tsp(start_city_index=0):
    """
    Risolve il TSP utilizzando l'algoritmo Greedy di Nearest Neighbor.
    :param start_city_index: Indice della città di partenza
    :return: Lista dell'ordine delle città visitate e lunghezza totale del percorso
    """
    dist_matrix=DIST_MATRIX.copy()
    visited = np.full(len(CITIES), False)  
    city = start_city_index  
    visited[city] = True

    tsp = [city]  

    while not np.all(visited):
        dist_matrix[:, city] = np.inf
        closest_city = np.argmin(dist_matrix[city])
        visited[closest_city] = True
        tsp.append(int(closest_city))
        city = closest_city

    tsp.append(start_city_index)
    
    total_distance = tsp_cost(tsp,DIST_MATRIX)
    
    return tsp, total_distance

In [20]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


In [21]:
def valid(tsp):
    """
    Verifica la validità di un percorso TSP.
    """
    tsp = np.array(tsp)
    
    if tsp[0] != tsp[-1]:
        return False
    
    # Verify that all cities are visited exactly once.
    cities_visited = tsp[:-1]
    expected_cities = np.arange(len(CITIES))
    
    return (len(np.unique(cities_visited)) == len(CITIES) and 
            np.all(np.isin(expected_cities, cities_visited)))
@counter
def tsp_cost(route, dist_matrix):
    """Calcola il costo del percorso e la sua validità."""
    route = np.array(route)
    # Calculates distances between consecutive cities.
    total_distance = np.sum([dist_matrix[route[i], route[i+1]] 
                           for i in range(len(route)-1)])
    return (total_distance, valid(route))

## Inversion mutation

In [22]:
def tweak(tsp):
    """Esegue un'inversion mutation su una soluzione TSP"""
    tsp = np.array(tsp)
    # Select two random points in the path to perform the inversion.
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
    # Reverses the sub-path between the two selected points.
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


##  Simulated Annealing (SA) by Initial Population.

In [23]:
def simulated_annealing_population(dist_matrix, initial_temp, cooling_rate, stop_temp, population_size=100):
    """Algoritmo di Simulated Annealing per generare una popolazione iniziale"""
    dist_matrix = np.array(dist_matrix)
     # Find an initial solution with the Greedy method.
    current_solution, current_cost = nearest_neighbor_tsp(start_city_index=0)
    best_solution = current_solution[:]
    best_cost = current_cost

    population = [(best_solution, best_cost)]
    temperature = initial_temp

    while temperature > stop_temp:
        new_solution = tweak(current_solution[:])
        new_cost = tsp_cost(new_solution, dist_matrix)

        if (new_cost[0] < current_cost[0] or 
            np.random.random() < np.exp((current_cost[0] - new_cost[0]) / temperature)):
            current_solution = new_solution
            current_cost = new_cost

            if new_cost[0] < best_cost[0]:
                best_solution = new_solution
                best_cost = new_cost
    
            population.append((current_solution, current_cost))
            
        temperature *= cooling_rate
    # Sort the population by cost and return the best 'population_size' individuals    
    population = np.array(population, dtype=object)
    sorted_indices = np.argsort([cost[0] for _, cost in population])
    population = population[sorted_indices]
    
    return population[:population_size].tolist()

In [24]:
def initial_population(pop_size):
    """Inizializza la popolazione usando parametri predefiniti."""
    initial_temp = 1000
    cooling_rate = 0.992
    stop_temp = 1e-200
    # Generate initial population with Simulated Annealing.
    return simulated_annealing_population(DIST_MATRIX, initial_temp,
                                        cooling_rate, stop_temp, pop_size)


##  Crossover Inver-Over

In [25]:
def inver_over_crossover(parent1, parent2, num_iterations=1):
    """Applica il crossover inver-over """
    parent1, parent2 = np.array(parent1), np.array(parent2)
    child = parent1.copy()
 # Perform a series of crossover iterations to create the child
    for _ in range(num_iterations):
        start_index = np.random.randint(1, len(child) - 2)
        start_city = child[start_index]
        # Randomly select a destination city from one of the parents.
        if np.random.random() < 0.5:
            valid_cities = child[1:-1][child[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        else:
            valid_cities = parent2[1:-1][parent2[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        # Get start and destination city locations in the son.
        start_pos = np.where(child == start_city)[0][0]
        end_pos = np.where(child == end_city)[0][0]
        
        if start_pos > end_pos:
            start_pos, end_pos = end_pos, start_pos
          # Reverse the sub-path between the two positions.
        child[start_pos:end_pos + 1] = np.flip(child[start_pos:end_pos + 1])
    # The child ends up in the same city as the starting city.
    if child[0] != child[-1]:
        child = np.append(child, child[0])
    
    return child.tolist()

In [26]:
def tournament_selection(population, k=5):
    """Selezione torneo usando NumPy."""
    # Randomly selects 'k' individuals from the population.
    selected_indices = np.random.choice(len(population), k, replace=False)
    selected = np.array(population, dtype=object)[selected_indices]
    # Returns the individual with the lowest cost among those selected.
    return min(selected, key=lambda x: x[1][0])  # x[1][0] accede al costo


## Inversion mutation

In [27]:
def inversion_mutation(tsp):
    """Esegue un'inversion mutation """
    tsp = np.array(tsp)
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


## EA with adaptive mutation and multiple restart with simulated anneling.

In [28]:


def genetic_algorithm_tsp_adaptive_mutation(
    
    dist_matrix,
    population_size=100, 
    tournament_size=10, 
    num_generations=10000, 
    initial_mutation_rate=0.1, 
    crossover_iterations=1,
    max_generations_without_improvement=500,
    mutation_increase_factor=1.5,
    mutation_decrease_factor=0.7,
    mutation_threshold=1.3, # Threshold for restarting
    initial_temp=1000,      
    cooling_rate=0.992,
    stop_temp=1e-200,

):
    """Risolve il TSP utilizzando un Algoritmo Genetico con mutazione adattiva."""
    dist_matrix = np.array(dist_matrix)
    population = initial_population(population_size)
    current_mutation_rate = initial_mutation_rate

    best_solution, best_cost = min(population, key=lambda x: x[1][0])
    all_time_best_solution = best_solution
    all_time_best_cost = best_cost
    generations_without_improvement = 0
    

    for generation in tqdm(range(num_generations)):
        # Check if it is necessary to reboot with SA.
        if current_mutation_rate > mutation_threshold:
            print(f"\nMutation rate ({current_mutation_rate:.3f}) exceeded threshold ({mutation_threshold})")
            print("Restarting with Simulated Annealing...")
            
             # Run SA and get new population
            new_population = simulated_annealing_population(
                dist_matrix, 
                initial_temp, 
                cooling_rate, 
                stop_temp, 
                population_size
            )
            population = sorted(new_population, key=lambda x: x[1][0])[:population_size]
             # Mutation rate reset
            current_mutation_rate = initial_mutation_rate
            print("Population refreshed and mutation rate reset")

        new_population = []

        while len(new_population) < population_size:
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            
            child = inver_over_crossover(parent1[0], parent2[0], crossover_iterations)
            
            if np.random.random() < current_mutation_rate:
                child = inversion_mutation(child)

            child_cost = tsp_cost(child, dist_matrix)
            new_population.append((child, child_cost))

        combined_population = np.array(new_population + population, dtype=object)
        sorted_indices = np.argsort([cost[0] for _, cost in combined_population])
        population = combined_population[sorted_indices][:population_size].tolist()

        new_best_solution, new_best_cost = min(population, key=lambda x: x[1][0])

         # Update the best result ever if needed
        if new_best_cost[0] < all_time_best_cost[0]:
            all_time_best_solution = new_best_solution
            all_time_best_cost = new_best_cost

        if new_best_cost[0] < best_cost[0]:
            best_solution, best_cost = new_best_solution, new_best_cost
            generations_without_improvement = 0
            current_mutation_rate = max(initial_mutation_rate, 
                                     current_mutation_rate * mutation_decrease_factor)
        else:
            generations_without_improvement += 1

        if generations_without_improvement > max_generations_without_improvement:
            current_mutation_rate *= mutation_increase_factor
            generations_without_improvement = 0

        if generation % 1000 == 0:
            print(f"Generazione {generation}: "
                  f"Costo migliore corrente = {best_cost[0]:.2f} | "
                  f"Costo migliore assoluto = {all_time_best_cost[0]:.2f} | "
                  f"Tasso di mutazione = {current_mutation_rate:.3f} | "
                  f"Valido = {best_cost[1]}")
    
    return all_time_best_cost


## Vanuatu

In [29]:

CITIES = pd.read_csv('./cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

best_cost=genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=0, 
)
ic(best_cost,tsp_cost.calls)



0it [00:00, ?it/s]
ic| best_cost: (np.float64(1345.5449564733112), np.True_)
    tsp_cost.calls: 58196


((np.float64(1345.5449564733112), np.True_), 58196)

For vanuatu just applying simulated anneling to get the best solution is enough

## Italy

In [39]:

CITIES = pd.read_csv('./cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    mutation_increase_factor=1.3,
    mutation_decrease_factor=0.7
)
ic(best_cost,tsp_cost.calls)


  0%|          | 13/10000 [00:00<02:52, 57.88it/s]

Generazione 0: Costo migliore corrente = 4436.03 | Costo migliore assoluto = 4436.03 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 1004/10000 [00:26<08:07, 18.45it/s]

Generazione 1000: Costo migliore corrente = 4258.07 | Costo migliore assoluto = 4258.07 | Tasso di mutazione = 0.130 | Valido = True


 20%|██        | 2004/10000 [01:22<07:31, 17.69it/s]

...

Generazione 8000: Costo migliore corrente = 4172.76 | Costo migliore assoluto = 4172.76 | Tasso di mutazione = 0.286 | Valido = True


 90%|█████████ | 9013/10000 [06:46<00:15, 65.24it/s]

Generazione 9000: Costo migliore corrente = 4172.76 | Costo migliore assoluto = 4172.76 | Tasso di mutazione = 0.483 | Valido = True


100%|██████████| 10000/10000 [07:02<00:00, 23.68it/s]
ic| best_cost: (np.float64(4172.7626139164095), np.True_)
    tsp_cost.calls: 14163920


((np.float64(4172.7626139164095), np.True_), 14163920)

## Russia

In [41]:

CITIES = pd.read_csv('./cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=genetic_algorithm_tsp_adaptive_mutation(DIST_MATRIX)
ic(best_cost,tsp_cost.calls)


  0%|          | 5/10000 [00:00<03:38, 45.73it/s]

Generazione 0: Costo migliore corrente = 35167.10 | Costo migliore assoluto = 35167.10 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 1007/10000 [00:32<04:14, 35.29it/s]

Generazione 1000: Costo migliore corrente = 34920.67 | Costo migliore assoluto = 34920.67 | Tasso di mutazione = 0.100 | Valido = True


 20%|██        | 2007/10000 [01:16<02:34, 51.74it/s]

...

Generazione 8000: Costo migliore corrente = 34146.28 | Costo migliore assoluto = 34146.28 | Tasso di mutazione = 0.100 | Valido = True


 90%|█████████ | 9003/10000 [06:24<00:24, 40.19it/s]

Generazione 9000: Costo migliore corrente = 34146.28 | Costo migliore assoluto = 34146.28 | Tasso di mutazione = 0.225 | Valido = True


100%|██████████| 10000/10000 [06:45<00:00, 24.66it/s]
ic| best_cost: (np.float64(34146.27869437402), np.True_)
    tsp_cost.calls: 16396704


((np.float64(34146.27869437402), np.True_), 16396704)

## Us

In [32]:

CITIES = pd.read_csv('./cities/us.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=20000)
ic(best_cost,tsp_cost.calls)

  0%|          | 9/20000 [00:00<08:36, 38.71it/s]

Generazione 0: Costo migliore corrente = 45867.54 | Costo migliore assoluto = 45867.54 | Tasso di mutazione = 0.100 | Valido = True


  5%|▌         | 1008/20000 [00:27<08:53, 35.60it/s]

Generazione 1000: Costo migliore corrente = 43361.78 | Costo migliore assoluto = 43361.78 | Tasso di mutazione = 0.100 | Valido = True


 10%|█         | 2006/20000 [01:17<08:06, 36.98it/s]

...

Population refreshed and mutation rate reset


 95%|█████████▌| 19007/20000 [13:15<00:27, 35.57it/s]

Generazione 19000: Costo migliore corrente = 41723.30 | Costo migliore assoluto = 41723.30 | Tasso di mutazione = 0.150 | Valido = True


100%|██████████| 20000/20000 [13:43<00:00, 24.30it/s]
ic| best_cost: (np.float64(41723.295155113214), np.True_)
    tsp_cost.calls: 4407372


((np.float64(41723.295155113214), np.True_), 4407372)

## China

In [33]:

CITIES = pd.read_csv('./cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=genetic_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=30000, 
)
ic(best_cost,tsp_cost.calls)


  0%|          | 3/30000 [00:00<23:02, 21.70it/s]

Generazione 0: Costo migliore corrente = 63962.92 | Costo migliore assoluto = 63962.92 | Tasso di mutazione = 0.100 | Valido = True


  3%|▎         | 1004/30000 [00:48<20:58, 23.03it/s]

Generazione 1000: Costo migliore corrente = 61604.59 | Costo migliore assoluto = 61604.59 | Tasso di mutazione = 0.100 | Valido = True


  7%|▋         | 2004/30000 [01:46<23:49, 19.59it/s]  

...

Generazione 28000: Costo migliore corrente = 52809.34 | Costo migliore assoluto = 52809.34 | Tasso di mutazione = 0.100 | Valido = True


 97%|█████████▋| 29002/30000 [39:15<01:20, 12.36it/s]

Generazione 29000: Costo migliore corrente = 52804.65 | Costo migliore assoluto = 52804.65 | Tasso di mutazione = 0.105 | Valido = True


100%|██████████| 30000/30000 [40:53<00:00, 12.23it/s]
ic| best_cost: (np.float64(52760.00314631018), np.True_)
    tsp_cost.calls: 7465568


((np.float64(52760.00314631018), np.True_), 7465568)

The proposed genetic algorithm addresses the traveling salesman problem (TSP) by integrating an adaptive mutation strategy and a restart mechanism based on Simulated Annealing (SA). Mutation is dynamically regulated: the rate increases if no improvement is observed for a given number of generations and decreases if progress is made, thus promoting a balance between exploration and exploitation.(Simulated anneling runs for a very short time allowing for a diverse initial population.)

When the mutation rate exceeds a predefined threshold, the algorithm initiates a restart phase via SA, regenerating the population to avoid convergence to local minima. The approach uses tournament selection, permutation-specific crossover techniques of the TSP (inver-over), and an inversion-based mutation. This combination allows the algorithm to progressively improve the quality of the solution.